加载txt文件，建立本地知识库

In [ ]:
%pip install -r requirements.txt -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
#######################
# 做一些初始化等准备工作  #
#######################

from dotenv import load_dotenv
import os
import sys

load_dotenv()  # 加载.env文件

openapi_key = os.getenv("OPENAI_API_KEY")  # 确保openapi_key已设置

if not openapi_key:
    print("[ERROR] OPENAI_API_KEY not set")
    sys.exit(1)

In [ ]:
#################
# 进行文档分词处理 #
#################

from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import SpacyTextSplitter


# 初始化目录加载器
# loader = DirectoryLoader(path='./txt/out', glob="**/*.txt", loader_cls=TextLoader)
loader = DirectoryLoader(path='./txt', glob="**/*.txt", loader_cls=TextLoader)

# 加载文档
documents = loader.load()

# 初始化分词器
text_splitter = SpacyTextSplitter(pipeline='zh_core_web_sm',chunk_size=1000,chunk_overlap=0)

# 文档分块
split_docs = text_splitter.split_documents(documents)

In [ ]:
print(len(split_docs))

In [ ]:
###################################################
# 1. 调用open ai的embedding模型，将分词向量化
# 2. 保存分词结果到 chroma_db
###################################################

import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# TODO: 这里可以切换成其他的embedding模型，比如 LlamaCppEmbeddings, HuggingFaceHubEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

# 保存分词结果到向量数据库Chroma
db = Chroma.from_documents(documents=split_docs, embedding=embeddings, persist_directory='./chroma_db')
db.persist()
db = None

In [ ]:


# from langchain.llms import OpenAI
# from langchain.chains.question_answering import load_qa_chain
# import os
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import Chroma

# embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

# # 初始化llm
# llm = OpenAI(temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))
# chain = load_qa_chain(llm=llm, chain_type='stuff')

# persist_directory = './chroma_db'
# db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

# query = "如何处理需要不同账户与端口登录的不同机器？"
# docs = db.similarity_search(query, 1)

In [ ]:


from langchain.llms import LlamaCpp
from langchain.chains.question_answering import load_qa_chain
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

# 初始化 llama.cpp 模型
llm = LlamaCpp(model_path="/Users/zhiwen/code/llama.cpp/zh-models/7B/ggml-model-q4_0.bin")

chain = load_qa_chain(llm=llm, chain_type='stuff')

# chroma加载上述embedding后的暂存的数据
db = Chroma(persist_directory='./chroma_db', embedding_function=embeddings)

# 示例的prompt
query = "如何处理需要不同账户与端口登录的不同机器？"

docs = db.similarity_search(query, 1)

In [ ]:
print(docs[0].page_content)

In [40]:
# llama_tokenize: too many tokens 的报错参考如下issue: https://github.com/ggerganov/llama.cpp/issues/385
chain.run(input_documents=docs, question=query)

llama_tokenize: too many tokens


RuntimeError: Failed to tokenize: text="b" Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nInstallation\n\nTopics\n\nInstallation\n\xe4\xbb\x8eGithub\xe8\x8e\xb7\xe5\x8f\x96Ansible\n\xe9\x9c\x80\xe8\xa6\x81\xe5\xae\x89\xe8\xa3\x85\xe4\xba\x9b\xe4\xbb\x80\xe4\xb9\x88\n\xe9\x80\x89\xe6\x8b\xa9\xe5\x93\xaa\xe4\xb8\x80\xe4\xb8\xaa\xe7\x89\x88\xe6\x9c\xac?\n\xe5\xaf\xb9\xe7\xae\xa1\xe7\x90\x86\xe4\xb8\xbb\xe6\x9c\xba\xe7\x9a\x84\xe8\xa6\x81\xe6\xb1\x82\n\xe5\xaf\xb9\xe6\x89\x98\xe7\xae\xa1\xe8\x8a\x82\xe7\x82\xb9\xe7\x9a\x84\xe8\xa6\x81\xe6\xb1\x82\n\xe5\xae\x89\xe8\xa3\x85\xe7\xae\xa1\xe7\x90\x86\xe4\xb8\xbb\xe6\x9c\xba\n\xe4\xbb\x8e\xe6\xba\x90\xe7\xa0\x81\xe8\xbf\x90\xe8\xa1\x8c\n\xe9\x80\x9a\xe8\xbf\x87Yum\xe5\xae\x89\xe8\xa3\x85\xe6\x9c\x80\xe6\x96\xb0\xe5\x8f\x91\xe5\xb8\x83\xe7\x89\x88\xe6\x9c\xac\n\xe9\x80\x9a\xe8\xbf\x87Apt (Ubuntu)\xe5\xae\x89\xe8\xa3\x85\xe6\x9c\x